In [149]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [150]:
# load train data
uid_train = pd.read_csv('../data/train/uid_train.txt',sep='\t',header=None,names=('uid','label'))
voice_train = pd.read_csv('../data/train/voice_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_train = pd.read_csv('../data/train/sms_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_train = pd.read_csv('../data/train/wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [151]:
# load test data
voice_test = pd.read_csv('../data/testB/voice_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_test = pd.read_csv('../data/testB/sms_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_test = pd.read_csv('../data/testB/wa_test_b.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [152]:
uid_test = pd.DataFrame({'uid':pd.unique(wa_test['uid'])})
uid_test.to_csv('./uid_test_b.txt',index=None)

In [153]:
# reset
voice = pd.concat([voice_train,voice_test],axis=0)
sms = pd.concat([sms_train,sms_test],axis=0)
wa = pd.concat([wa_train,wa_test],axis=0)

**voice records**

In [154]:
voice_cnt = voice.groupby(['uid'])['in_out'].agg({'voice_count':'count'}).reset_index()
voice_unique_num_cnt = voice.groupby(['uid'])['opp_num'].agg({'voice_opp_num_unique_count': lambda x: len(pd.unique(x))}).reset_index()
# voice_unique_head_cnt = voice.groupby(['uid'])['opp_head'].agg({'voice_opp_head_unique_count': lambda x: len(pd.unique(x))}).reset_index()


In [155]:
voice_onehot_len_cnt = voice.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_opp_len_').reset_index().fillna(0)
voice_onehot_type_cnt = voice.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_call_type_').reset_index().fillna(0)
voice_onehot_inout_cnt = voice.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('voice_in_out_').reset_index().fillna(0)

voice_onehot_head_cnt = voice.groupby(['uid','opp_head'])['uid'].count().unstack().add_prefix('voice_opp_head_').reset_index().fillna(0)


In [156]:
voice['start_day'] = (voice.start_time.astype(int) / 1000000).astype(int)
voice['start_hour'] = (voice.start_time.astype(int) % 1000000 / 10000).astype(int)
voice['start_minute'] = (voice.start_time.astype(int) % 10000 / 100).astype(int)
voice['start_second'] = voice.start_time.astype(int) % 100

voice['end_day'] = (voice.end_time.astype(int) / 1000000).astype(int)
voice['end_hour'] = (voice.end_time.astype(int) % 1000000 / 10000).astype(int)
voice['end_minute'] = (voice.end_time.astype(int) % 10000 / 100).astype(int)
voice['end_second'] = voice.end_time.astype(int) % 100

voice['voice_dura'] = (voice.end_day-voice.start_day)*24*60*60+(voice.end_hour-voice.start_hour)*60*60+(voice.end_minute-voice.start_minute)*60+(voice.end_second-voice.start_second)

In [157]:
voice_dura_val = voice.groupby(['uid'])['voice_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('voice_dura_').reset_index()
voice_onehot_hour_cnt = voice.groupby(['uid','start_hour'])['uid'].count().unstack().add_prefix('voice_start_hour_').reset_index().fillna(0)


**sms records**

In [158]:
sms_cnt = sms.groupby(['uid'])['in_out'].agg({'sms_count':'count'}).reset_index()
sms_unique_num_cnt = sms.groupby(['uid'])['opp_num'].agg({'sms_opp_num_unique_count': lambda x: len(pd.unique(x))}).reset_index()
# sms_unique_head_cnt = sms.groupby(['uid'])['opp_head'].agg({'sms_opp_head_unique_count': lambda x: len(pd.unique(x))}).reset_index()


In [159]:
sms_onehot_len_cnt = sms.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_opp_len_').reset_index().fillna(0)
sms_onehot_inout_cnt = sms.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('sms_in_out_').reset_index().fillna(0)

sms_onehot_head_cnt = sms.groupby(['uid','opp_head'])['uid'].count().unstack().add_prefix('sms_opp_head_').reset_index().fillna(0)


In [160]:
sms['start_day'] = (sms.start_time.astype(int) / 1000000).astype(int)
sms['start_hour'] = (sms.start_time.astype(int) % 1000000 / 10000).astype(int)
sms['start_minute'] = (sms.start_time.astype(int) % 10000 / 100).astype(int)
sms['start_second'] = sms.start_time.astype(int) % 100

sms_onehot_hour_cnt = sms.groupby(['uid','start_hour'])['uid'].count().unstack().add_prefix('sms_start_hour_').reset_index().fillna(0)


**wa records** 

In [161]:
wa_unique_name_cnt = wa.groupby(['uid'])['wa_name'].agg({'wa_name_unique_count': lambda x: len(pd.unique(x))}).reset_index()


In [162]:
wa_onehot_type_cnt = wa.groupby(['uid','wa_type'])['uid'].count().unstack().add_prefix('wa_type_').reset_index().fillna(0)


In [163]:
# wa_visit_cnt_val = wa.groupby(['uid'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_').reset_index()
# wa_visit_dura_val = wa.groupby(['uid'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_dura_').reset_index()
# wa_up_flow_val = wa.groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_').reset_index()
# wa_down_flow_val = wa.groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_').reset_index()


In [164]:
wa['wa_mean_dura_per_visit'] = wa.visit_dura / wa.visit_cnt
wa_dura_per_visit_val = wa.groupby(['uid'])['wa_mean_dura_per_visit'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_dura_per_visit_').reset_index()


In [165]:
for i in range(45):
    up_flow = wa[wa.date == str(i+1)].groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('up_in_date'+str(i+1)+'_').reset_index().fillna(0)
    down_flow = wa[wa.date == str(i+1)].groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('down_in_date'+str(i+1)+'_').reset_index().fillna(0)
    wa_dura_per_visit_val = pd.merge(wa_dura_per_visit_val, up_flow, how='left', on='uid')
    wa_dura_per_visit_val = pd.merge(wa_dura_per_visit_val, down_flow, how='left', on='uid')


,uid,wa_dura_per_visit_std,wa_dura_per_visit_max,wa_dura_per_visit_min,wa_dura_per_visit_median,wa_dura_per_visit_mean,wa_dura_per_visit_sum,up_in_date1_std,up_in_date1_max,up_in_date1_min,...,up_in_date45_min,up_in_date45_median,up_in_date45_mean,up_in_date45_sum,down_in_date45_std,down_in_date45_max,down_in_date45_min,down_in_date45_median,down_in_date45_mean,down_in_date45_sum
0,u0001,1143.680091,9150.000000,0.0,1130.000000,1236.558231,1.428225e+06,0.0,0.0,0.0,...,52.0,17877.5,2.253695e+05,8564041.0,1.240819e+06,6.273218e+06,40.0,42964.5,4.584999e+05,1.742300e+07
1,u0002,1478.449461,5112.000000,0.0,2084.285714,1880.592515,5.453718e+04,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00
2,u0003,1146.996555,8092.000000,0.0,956.000000,1176.050697,1.372451e+06,0.0,0.0,0.0,...,60.0,23884.0,4.398688e+05,25072520.0,6.162219e+07,4.570219e+08,120.0,70849.0,1.267444e+07,7.224432e+08
3,u0004,1387.195407,9553.000000,0.0,1065.631628,1303.624458,2.190089e+06,0.0,0.0,0.0,...,452.0,9644.0,7.722000e+04,2471040.0,4.922232e+05,2.655040e+06,319.0,16378.0,1.897105e+05,6.070737e+06
4,u0005,1340.392078,8223.000000,0.0,990.000000,1250.471406,8.015522e+05,0.0,0.0,0.0,...,0.0,2023.5,5.088625e+03,81418.0,1.890868e+05,7.609150e+05,240.0,3070.0,5.690719e+04,9.105150e+05
5,u0006,1282.338173,7145.500000,0.0,699.333333,1079.816067,5.237108e+05,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00
6,u0007,1169.570567,8149.000000,0.0,904.588235,1163.879750,1.531666e+06,0.0,0.0,0.0,...,77.0,9091.0,2.543332e+04,635833.0,9.289837e+05,4.661579e+06,0.0,15501.0,2.229975e+05,5.574938e+06
7,u0008,1278.574532,9370.000000,0.0,920.173913,1184.599394,1.265152e+06,0.0,0.0,0.0,...,160.0,4162.0,8.500883e+04,2465256.0,3.910391e+06,2.093690e+07,40.0,15180.0,8.836665e+05,2.562633e+07
8,u0009,1298.202756,9453.000000,0.0,994.250000,1236.003449,1.320052e+06,0.0,0.0,0.0,...,200.0,4098.5,9.369100e+03,93691.0,1.054180e+05,3.423690e+05,0.0,6532.0,4.458790e+04,4.458790e+05
9,u0010,1082.201412,4251.000000,0.0,617.062500,1058.292467,9.207144e+04,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00


In [167]:
wa_dura_per_visit_val = wa_dura_per_visit_val.fillna(0)
wa_dura_per_visit_val.head(5)

,uid,wa_dura_per_visit_std,wa_dura_per_visit_max,wa_dura_per_visit_min,wa_dura_per_visit_median,wa_dura_per_visit_mean,wa_dura_per_visit_sum,up_in_date1_std,up_in_date1_max,up_in_date1_min,...,up_in_date45_min,up_in_date45_median,up_in_date45_mean,up_in_date45_sum,down_in_date45_std,down_in_date45_max,down_in_date45_min,down_in_date45_median,down_in_date45_mean,down_in_date45_sum
0,u0001,1143.680091,9150.0,0.0,1130.000000,1236.558231,1.428225e+06,0.0,0.0,0.0,...,52.0,17877.5,225369.50000,8564041.0,1.240819e+06,6273218.0,40.0,42964.5,4.584999e+05,17422996.0
1,u0002,1478.449461,5112.0,0.0,2084.285714,1880.592515,5.453718e+04,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0
2,u0003,1146.996555,8092.0,0.0,956.000000,1176.050697,1.372451e+06,0.0,0.0,0.0,...,60.0,23884.0,439868.77193,25072520.0,6.162219e+07,457021859.0,120.0,70849.0,1.267444e+07,722443226.0
3,u0004,1387.195407,9553.0,0.0,1065.631628,1303.624458,2.190089e+06,0.0,0.0,0.0,...,452.0,9644.0,77220.00000,2471040.0,4.922232e+05,2655040.0,319.0,16378.0,1.897105e+05,6070737.0
4,u0005,1340.392078,8223.0,0.0,990.000000,1250.471406,8.015522e+05,0.0,0.0,0.0,...,0.0,2023.5,5088.62500,81418.0,1.890868e+05,760915.0,240.0,3070.0,5.690719e+04,910515.0


In [168]:
feature = [
    voice_cnt,
    voice_unique_num_cnt,
#     voice_unique_head_cnt,
    voice_onehot_len_cnt,
    voice_onehot_type_cnt,
    voice_onehot_inout_cnt,
    voice_dura_val,
    voice_onehot_head_cnt,
    sms_cnt,
    sms_unique_num_cnt,
#     sms_unique_head_cnt,
    sms_onehot_len_cnt,
    sms_onehot_inout_cnt,
    sms_onehot_head_cnt,
    sms_onehot_hour_cnt,
    wa_unique_name_cnt,
    wa_onehot_type_cnt,
#     wa_visit_cnt_val,
#     wa_visit_dura_val,
#     wa_up_flow_val,
#     wa_down_flow_val,
    wa_dura_per_visit_val
]

In [169]:
train_feature = uid_train
for feat in feature:
    train_feature=pd.merge(train_feature,feat,how='left',on='uid')

test_feature = uid_test
for feat in feature:
    test_feature=pd.merge(test_feature,feat,how='left',on='uid')

In [170]:
train_feature.to_csv('./train_feature.csv',index=None)
test_feature.to_csv('./test_feature.csv',index=None)

In [171]:
### MODEL ###

In [172]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [173]:
train = pd.read_csv('./train_feature.csv')
test = pd.read_csv('./test_feature.csv')

In [174]:
dtrain = lgb.Dataset(train.drop(['uid','label'],axis=1),label=train.label)
dtest = lgb.Dataset(test.drop(['uid'],axis=1))

In [175]:
lgb_params =  {
    'boosting_type': 'gbdt',
    'objective': 'binary',
#    'metric': ('multi_logloss', 'multi_error'),
    #'metric_freq': 100,
    'is_training_metric': False,
    'min_data_in_leaf': 12,
    'num_leaves': 64,
    'learning_rate': 0.08,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'verbosity':-1,
#    'gpu_device_id':2,
#    'device':'gpu'
#    'lambda_l1': 0.001,
#    'skip_drop': 0.95,
#    'max_drop' : 10
    #'lambda_l2': 0.005
    #'num_threads': 18
}    

In [176]:
def evalMetric(preds,dtrain):
    label = dtrain.get_label()
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    auc = metrics.roc_auc_score(pre.label,pre.preds)
    pre.preds=pre.preds.map(lambda x: 1 if x>=0.5 else 0)
    f1 = metrics.f1_score(pre.label,pre.preds)
    res = 0.6*auc +0.4*f1
    return 'res',res,True

In [177]:
lgb.cv(lgb_params,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=5,num_boost_round=10000,nfold=3,metrics=['evalMetric'])

[5]	cv_agg's res: 0.803424 + 0.00629722
[10]	cv_agg's res: 0.816045 + 0.00828169
[15]	cv_agg's res: 0.821807 + 0.00568816
[20]	cv_agg's res: 0.827651 + 0.00800642
[25]	cv_agg's res: 0.826644 + 0.0082242
[30]	cv_agg's res: 0.830683 + 0.00726334
[35]	cv_agg's res: 0.833012 + 0.00390066
[40]	cv_agg's res: 0.8313 + 0.00700245
[45]	cv_agg's res: 0.835244 + 0.00578985
[50]	cv_agg's res: 0.83654 + 0.00590477
[55]	cv_agg's res: 0.838027 + 0.00941796
[60]	cv_agg's res: 0.839234 + 0.00823977
[65]	cv_agg's res: 0.839003 + 0.00828127
[70]	cv_agg's res: 0.839421 + 0.00639525
[75]	cv_agg's res: 0.838165 + 0.0076883
[80]	cv_agg's res: 0.839585 + 0.00927351
[85]	cv_agg's res: 0.840382 + 0.00931656
[90]	cv_agg's res: 0.840001 + 0.00899178
[95]	cv_agg's res: 0.843012 + 0.0100838
[100]	cv_agg's res: 0.844183 + 0.00953654
[105]	cv_agg's res: 0.843474 + 0.00934592
[110]	cv_agg's res: 0.84265 + 0.00899834
[115]	cv_agg's res: 0.84196 + 0.00852801
[120]	cv_agg's res: 0.845122 + 0.0097641
[125]	cv_agg's res: 0

{'res-mean': [0.73758529419724006,
  0.77712899680133629,
  0.78966011114977785,
  0.79685366016807524,
  0.80342381926183248,
  0.80930285602470431,
  0.8072382554848837,
  0.81425871093032942,
  0.81353700189500378,
  0.81604481308797194,
  0.81645697527014394,
  0.81948297775411705,
  0.81959451541078732,
  0.82000050396523616,
  0.82180736282269484,
  0.82128132907124396,
  0.82610912043645879,
  0.82657975132940031,
  0.82695912495363189,
  0.82765068894658178,
  0.82585287799008966,
  0.82761795059887888,
  0.826690308064018,
  0.82804962716034047,
  0.82664420370107417,
  0.82999034666539606,
  0.82795746691527849,
  0.82919327026542788,
  0.83087109097415002,
  0.83068272865041226,
  0.8305202355322171,
  0.830129791301221,
  0.83276164224474059,
  0.83275034143830595,
  0.83301157109003121,
  0.83320984731775327,
  0.83226493581700967,
  0.83430360402876114,
  0.83238150739590555,
  0.831299998420581,
  0.83202984122805101,
  0.83399851732015762,
  0.83468351930009865,
  0.834

In [178]:
model =lgb.train(lgb_params,dtrain,feval=evalMetric,verbose_eval=5,num_boost_round=300,valid_sets=[dtrain])

[5]	training's res: 0.942264
[10]	training's res: 0.952162
[15]	training's res: 0.96358
[20]	training's res: 0.97234
[25]	training's res: 0.97922
[30]	training's res: 0.985272
[35]	training's res: 0.989897
[40]	training's res: 0.992769
[45]	training's res: 0.996554
[50]	training's res: 0.997744
[55]	training's res: 0.999105
[60]	training's res: 0.999777
[65]	training's res: 0.999778
[70]	training's res: 0.999778
[75]	training's res: 1
[80]	training's res: 1
[85]	training's res: 1
[90]	training's res: 1
[95]	training's res: 1
[100]	training's res: 1
[105]	training's res: 1
[110]	training's res: 1
[115]	training's res: 1
[120]	training's res: 1
[125]	training's res: 1
[130]	training's res: 1
[135]	training's res: 1
[140]	training's res: 1
[145]	training's res: 1
[150]	training's res: 1
[155]	training's res: 1
[160]	training's res: 1
[165]	training's res: 1
[170]	training's res: 1
[175]	training's res: 1
[180]	training's res: 1
[185]	training's res: 1
[190]	training's res: 1
[195]	trainin

In [179]:
pred = model.predict(test.drop(['uid'],axis=1))
res = pd.DataFrame({'uid':test.uid,'label':pred})
res = res.sort_values(by='label',ascending=False)
res.label = res.label.map(lambda x: 1 if x>=0.5 else 0)
res.label = res.label.map(lambda x: int(x))

res.to_csv('./result.csv',index=False,header=False,sep=',',columns=['uid','label'])